<a href="https://colab.research.google.com/github/kien286/A-Love-Survey-Analysis/blob/main/strategy_return.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
from google.colab import drive

drive.mount('/content/gdrive/')

import os
os.chdir('/content/gdrive/My Drive/polymarket_arbitrage')

import sys
sys.path.append('/content/gdrive/My Drive/polymarket_arbitrage')

import pandas as pd
import numpy as np

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [124]:
folder = os.listdir('backfill_1')

In [197]:
filepath = folder[43]
df = pd.read_csv(f'backfill_1/{filepath}')
df['timestamp'] = pd.to_datetime(df['t']*10**9)
df

,t,p,timestamp
0,1700248562,0.5000,2023-11-17 19:16:02
1,1700248622,0.5000,2023-11-17 19:17:02
2,1700248682,0.5000,2023-11-17 19:18:02
3,1700248742,0.5000,2023-11-17 19:19:02
4,1700248802,0.5000,2023-11-17 19:20:02
...,...,...,...
7813,1700717402,0.0015,2023-11-23 05:30:02
7814,1700717462,0.0015,2023-11-23 05:31:02
7815,1700717522,0.0015,2023-11-23 05:32:02
7816,1700717582,0.0015,2023-11-23 05:33:02


In [198]:
filepath

'will-vvd-win-a-plurality-in-the-2023-dutch-general-election.Yes.is-winner-False.csv.gz'

In [202]:
import pandas as pd

def calculate_return(df, column='p'):
    """
    Calculates the return based on the rules specified, traversing column `p`.

    Args:
        df (pd.DataFrame): The input DataFrame with a column `p`.
        column (str): The name of the column to process (default is 'p').

    Returns:
        float: The calculated return based on the rules.
    """
    p_series = df[column]
    variable = None  # Keeps track of 'Yes' or 'No'
    record_value = 0  # To track the recorded value
    pattern_count = 0  # Count pattern flips
    return_value = 0  # Final return value
    cost_value = 0

    for idx, p in enumerate(p_series):
        # Rule 1: Search for the first value outside the range 0.45 to 0.55
        if variable is None and (p < 0.45 or p > 0.55):
          if p < 0.45:
            variable =  "No"
            cost_value = 0.55
            record_value = 0.45

          elif p > 0.55:
            variable = 'Yes'
            cost_value = 0.55
            record_value = 0.45

          continue

        # Rule 2: Handle pattern changes
        if variable == "Yes" and p < 0.45:

            pattern_count += 1
            if pattern_count <= 4:
              cost_value += 0.1
            print('-------------------')
            print(p)
            print(record_value)
            print(cost_value)
            record_value = 0.45
            variable = "No"

        elif variable == "No" and p > 0.55:

            pattern_count += 1
            if pattern_count <= 4:
              cost_value += 0.1
            print('-------------------')
            print(p)
            print(record_value)
            print(cost_value)
            record_value = 0.55
            variable = "Yes"


        # Rule 3: Stop process after 5 pattern changes
        if pattern_count > 4:
            return_value = 0.45
            break

    # Rule 4: Final Return Decision
    if pattern_count <= 4:
        if p < 0.5 and variable == "No":
            return_value = 1
        elif p > 0.5 and variable == "Yes":
            return_value = 1
        else:
            return_value = 0

    return return_value - cost_value

result = calculate_return(df, column='p')
print("Calculated Return:", result)


-------------------
0.445
0.45
0.65
-------------------
0.63
0.45
0.75
-------------------
0.375
0.55
0.85
-------------------
0.615
0.45
0.95
-------------------
0.35
0.55
0.95
Calculated Return: -0.49999999999999994
